In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 56.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=0a65f87f08605291cde152d80f8de925e597433d4143ddcfb324848eae927260
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SQLContext, Row, SparkSession

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import math
import time

In [5]:
# Getting the SparkContext
conf = SparkConf().setMaster("local").setAppName("count")
sc = SparkContext.getOrCreate(conf=conf)
# Initializing the SQLContext
sqlContext = SQLContext(sc)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [6]:
links_count = 6012
beta = 0.85
epsilon = 0.01
url = [i+1 for i in range(links_count)]

In [18]:
set_url= sc.parallelize(url)
ranks = set_url.map(lambda x: (x,1/links_count)).cache() #(id, pagerank_score)
links = sc.textFile("drive/My Drive/STUDY/Big data/hollins_links.dat")\
          .map(lambda l: l.split(' '))\
          .map(lambda x: (int(x[0]),int(x[1]))) #(url_id_1, url_id_2) 
link_out = links.groupByKey().mapValues(list).cache() #(url_id, [list_link_out])
link_in = links.map(lambda x: x[1]).distinct() #(url has link in)
url_has_no_link_in = set_url.subtract(link_in).map(lambda x: (x,0)).cache()

start = time.time()
eps=0.2
i=1
while (i<15) and (eps > epsilon):
  start_ = time.time()
  rold = ranks

  #Tính r'new
  r_new = link_out.join(ranks).values()\
                  .flatMap(lambda x: [(i,beta*x[1]/len(x[0])) for i in x[0]] )\
                  .reduceByKey(lambda x,y: x+y) 
  #ranks.unpersist()
  #Tính phần xác suất bị mất
  s = r_new.values().sum() 
  leak = (1-s)/links_count 

  ranks = r_new.union(url_has_no_link_in).mapValues(lambda x: x+leak)
  eps = ranks.join(rold).values().map(lambda x: abs(x[0] - x[1])).sum()
  
  print("Eps: ",eps)
  #print(ranks.takeOrdered(5, key=lambda x: -x[1]))
  print("Time",i,":",time.time() - start_)
  i+=1

print("Time: ",time.time() - start)

Eps:  0.4907346158919128
Time 1 : 1.640383005142212
Eps:  0.2001414571754684
Time 2 : 1.6827895641326904
Eps:  0.10807245253066153
Time 3 : 2.514404535293579
Eps:  0.0667821351712067
Time 4 : 3.7230637073516846
Eps:  0.04340338585792096
Time 5 : 4.167089462280273
Eps:  0.029440978484413557
Time 6 : 5.321201801300049
Eps:  0.020446198053264995
Time 7 : 7.456078052520752
Eps:  0.014266703187545172
Time 8 : 8.57223129272461
Eps:  0.010466823189697598
Time 9 : 10.52560305595398
Eps:  0.007672454147727572
Time 10 : 11.537799835205078
Time:  57.148226261138916


In [19]:
ranks.values().sum()

1.000000000000001

In [20]:
ranks.takeOrdered(10, key=lambda x: -x[1])

[(2, 0.020342191497028476),
 (37, 0.009487375773207355),
 (38, 0.008793044100898852),
 (61, 0.008237781057141935),
 (52, 0.008202176317816513),
 (43, 0.007310230608262429),
 (425, 0.0067090378160536614),
 (27, 0.006121903841048367),
 (28, 0.0057035523731509),
 (29, 0.004470489627716423)]